In [1]:
import os
import cv2
import time
import imutils
import mahotas as mt
import numpy as np
from sklearn import metrics
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from skimage.feature import greycomatrix,greycoprops,hog
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
#glcm features

def extract_glcm_features(image_patch):
    
    #feature extraction using GLCM
    
    grey_patch=cv2.cvtColor(image_patch,cv2.COLOR_BGR2GRAY)
    glcm_mat=greycomatrix(grey_patch,[1],[0,np.pi/4,np.pi,0.75*np.pi],levels=256,symmetric=True,normed=True)
    #different features
    contrast=np.mean(greycoprops(glcm_mat,'contrast'))
        
    dissimilar=np.mean(greycoprops(glcm_mat,'dissimilarity'))
        
    homogeneity=np.mean(greycoprops(glcm_mat,'homogeneity'))
        
    energy=np.mean(greycoprops(glcm_mat,'energy'))
        
    asm=np.mean(greycoprops(glcm_mat,'ASM'))
        
    correlation=np.mean(greycoprops(glcm_mat,'correlation'))
        
    feature_vect=[correlation,dissimilar,homogeneity,energy,asm,correlation]
    
    
    return feature_vect

In [6]:
#making the train dataset along with labels

gt_folder_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Training01\Ground Truth'

orig_folder_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Training01\Original Image'

#segments = ['Bicycle', 'Car', 'Human']
segments = ['Car', 'Human']

#lists of data
train_label = []
glcm_train_data = []
hog_train_data = []
brisk_train_data = []

humoments_train_data = []
haralick_train_data = []
#encoding the label names into numbers
#human is 0
#car is 1
#bicycle is 2

encoded ={'human':0, 'car':1, 'bicycle':1}


#describing the HOG descriptor
hog = cv2.HOGDescriptor()

#initialising the brisk descriptor
brisk = cv2.BRISK_create()


for segment in segments:
    
    image_folder = os.path.join(orig_folder_path,segment)
    gt_image_folder = os.path.join(gt_folder_path,segment)
    
    
    list_orig = os.listdir(image_folder)
    list_gt = os.listdir(gt_image_folder)
    
    for orig_name,gt_name in zip(list_orig,list_gt):
        
        orig_image = cv2.imread(os.path.join(image_folder,orig_name),1)
        gt_image = cv2.imread(os.path.join(gt_image_folder,gt_name),0)
        
        if segment == 'Bicycle':
            gt_image = np.invert(gt_image)
            train_label.append(1)
        
        if segment == 'Human':
            train_label.append(0)
            
        if segment == 'Car':
            train_label.append(1)
            
        #edged = imutils.auto_canny(gt_image)    
        contours,hierarchies = cv2.findContours(gt_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        max_cnt=max(contours,key=cv2.contourArea)
            
        max_rect=cv2.boundingRect(max_cnt)
        x,y,w,h = max_rect
        
        '''
        cv2.rectangle(orig_image,(x,y),(x+w,y+h),(0,0,255),1)
        
        cv2.imshow(orig_name,orig_image)
        k=cv2.waitKey(1) & 0xFF
        
        if k == ord('q'):
            cv2.destroyWindow(orig_name)
            break
        else:
            cv2.destroyWindow(orig_name)
         
        #cv2.destroyWindow(orig_name)
        '''
        
        image_patch=orig_image[y:y+h, x:x+w ]
        #resizing to 64 X 128
        #print(orig_name,x,y,w,h)
        
        resized_patch = cv2.resize(image_patch,(64,128))
        
        edged_patch = cv2.Canny(resized_patch,200,230)
        
        '''
        cv2.imshow('edge',edged_patch)
        cv2.imshow('orig',image_patch)
        k=cv2.waitKey(0) & 0xFF
        
        if k == ord('q'):
            cv2.destroyWindow(orig_name)
            cv2.destroyWindow('edge')
            cv2.destroyWindow('orig')
            break
        else:
            cv2.destroyWindow(orig_name)
            cv2.destroyWindow('edge')
            cv2.destroyWindow('orig')
        '''
        
        #extracting features
        #extracting the glcm features
        
        glcm_train_data.append(np.asarray(extract_glcm_features(resized_patch)))
        
        #extract hog features
        hog_fetures = hog.compute(cv2.cvtColor(resized_patch,cv2.COLOR_BGR2GRAY))
        hog_train_data.append(hog_fetures.reshape(3780))
        
        humoments_train_data.append(cv2.HuMoments(cv2.moments(gt_image[y:y+h,x:x+w])))
        
        haralick_train_data.append(mt.features.haralick(resized_patch))
        '''
        #extract brisk features
        keypoint , features = brisk.detectAndCompute(resized_patch,None)
        brisk_train_data.append(features.flatten())
        '''
        


In [7]:
# randomise the datasets: labels and data
randomise = np.arange(len(hog_train_data))
np.random.shuffle(randomise)

hog_train_data = np.asarray(hog_train_data)
hog_train_data = hog_train_data[randomise]

glcm_train_data = np.asarray(glcm_train_data)
glcm_train_data = glcm_train_data[randomise]

'''
humoments_train_data = np.asarray(humoments_train_data)
humoments_train_data = humoments_train_data.reshape(-1,7)
humoments_train_data = humoments_train_data[randomise]


haralick_train_data = np.asarray(haralick_train_data)
haralick_train_data = haralick_train_data.reshape(248,-1)
haralick_train_data = haralick_train_data[randomise]

'''
train_label = np.asarray(train_label)
train_label = train_label[randomise]

In [23]:
linsvm=SVC(kernel='linear',gamma='auto')
polysvm = SVC(kernel = 'poly',gamma='auto')
rbfsvm = SVC(kernel='rbf',gamma='auto')

'''
linsvm.fit(haralick_train_data,train_label)
polysvm.fit(haralick_train_data, train_label)
rbfsvm.fit(haralick_train_data,train_label)
print()


linsvm.fit(humoments_train_data,train_label)
polysvm.fit(humoments_train_data, train_label)
rbfsvm.fit(humoments_train_data,train_label)
print()

linsvm.fit(hog_train_data,train_label)
polysvm.fit(hog_train_data, train_label)
rbfsvm.fit(hog_train_data,train_label)

'''
linsvm.fit(glcm_train_data,train_label)
polysvm.fit(glcm_train_data, train_label)
rbfsvm.fit(glcm_train_data,train_label)


orig_test_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Testing01\Original'
gt_test_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Testing01\GT'

#gt_test_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Training01\Ground Truth\Car'
#orig_test_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Training01\Original Image\Car'

list_orig_test = os.listdir(orig_test_path)
list_gt_test = os.listdir(gt_test_path)

#print(list_orig_test)
#print(list_gt_test)
bbox_encoding = {0:(0,0,255), 1:(0,255,0), 2:(255,0,0)}

hog=cv2.HOGDescriptor()

for orig_test_image_name,gt_test_image_name in zip(list_orig_test,list_gt_test):
    
    orig_test_image = cv2.imread(os.path.join(orig_test_path,orig_test_image_name),1)
    #print(os.path.join(orig_test_path,orig_test_image_name))
    
    gt_test_image = cv2.imread(os.path.join(gt_test_path,gt_test_image_name),0)
    #gt_test_image = np.invert(gt_test_image)
    #print(os.path.join(gt_test_path,gt_test_image_name))
    
    
    contours,hierarchies = cv2.findContours(gt_test_image , cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    rects = [cv2.boundingRect(cnt) for cnt in contours]
    
    patches = []
    
    for rect in rects:
        x,y,w,h = rect
        patch = orig_test_image[y:y+h,x:x+w]
        resized_patch = cv2.resize(patch,(64,128))
        
        glcm_feature = np.asarray(extract_glcm_features(resized_patch))
        #print(glcm_feature.shape)
        hog_features=hog.compute(resized_patch).reshape(3780)
        
        humoment_data = cv2.HuMoments(cv2.moments(gt_test_image[y:y+h,x:x+w]))
        humoment_data = humoment_data.reshape(-1,7)
        
        haralick_feature = mt.features.haralick(resized_patch)
        haralick_feature = haralick_feature.reshape(1,-1)
        print(haralick_feature.shape)
        
        y_pred = rbfsvm.predict([glcm_feature])
        
        #print(y_pred[0])
        
        cv2.rectangle(orig_test_image, (x,y),(x+w,y+h),bbox_encoding[y_pred[0]],1)
    cv2.imshow(orig_test_image_name,orig_test_image)
    k = cv2.waitKey(0) & 0xFF
    if k== ord('q'):
        cv2.destroyWindow(orig_test_image_name)
        break
    else :
        cv2.destroyWindow(orig_test_image_name)


(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)
(1, 169)


In [10]:

linsvm=SVC(kernel='linear',gamma='auto')
polysvm = SVC(kernel = 'poly',gamma='auto')
rbfsvm = SVC(kernel='rbf',gamma='auto')


linsvm.fit(hog_train_data,train_label)
polysvm.fit(hog_train_data, train_label)
rbfsvm.fit(hog_train_data,train_label)



#testing scene

frameset_test_path = r'C:\Users\Legion\Desktop\Internship_IIT\ethz\images\pedxing-seq1'

list_frames = os.listdir(frameset_test_path)

list_frames.remove('maps')

fg_extractor = cv2.createBackgroundSubtractorKNN()

bbox_encoding = { 0:(0,0,255), 1:(0,255,0) }

for frame_name in list_frames:
    
    frame_path = os.path.join(frameset_test_path, frame_name)
    
    frame =cv2.imread(frame_path, 1)
    frame = cv2.resize(frame,(200,150))
    grey_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    fg_mask = fg_extractor.apply(grey_frame)
    
    fg_mask = cv2.GaussianBlur(fg_mask, (5,5), 0)
    
    contours,hierarchies = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    
    for cnt in contours:
        
        x,y,w,h = cv2.boundingRect(cnt)
        if w*h  > 100:
            
            patch = frame[y:y+h,x:x+w]
            resized_patch = cv2.resize(patch,(64,128))
        
            glcm_feature = np.asarray(extract_glcm_features(resized_patch))
            #print(glcm_feature.shape)
            hog_features=hog.compute(resized_patch).reshape(3780)
        
            y_pred = rbfsvm.predict([hog_features])
        
            #print(y_pred[0])
        
            cv2.rectangle(frame, (x,y),(x+w,y+h),bbox_encoding[y_pred[0]],1)
    cv2.imshow('mask',fg_mask)
    cv2.imshow('grey_image', cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    cv2.imshow('frame_name',frame)
    
    k = cv2.waitKey(1) & 0xFF
    time.sleep(0.0003333)
    if k== ord('q'):        
        break
        
cv2.destroyAllWindows()    
    

In [9]:
import time

In [1]:
import sknn

ModuleNotFoundError: No module named 'sknn'

In [2]:
import sknn